<a href="https://colab.research.google.com/github/purnachand1186/LLM_Engineering/blob/main/Diagnose_Llama_Access_issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diagnose common problems with HuggingFace access

Please step through this colab to figure out the issue.

By this point, you should have pressed the Key symbol in the sidebar to the left, and entered a secret with:  

Key: `HF_TOKEN`  
Value: your actual token, starting hf_.

And the switch next to it should be turned on.

See bottom of this colab if you've not been approved for Meta.

In [ ]:
import requests
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer

In [ ]:
hf_token = userdata.get('HF_TOKEN')
if hf_token.startswith("hf_"):
  print(f"HF Token has the right prefix and begins -->{hf_token[:5]}<--")
else:
  print("HF Token not found - please use the key icon in the left sidebar to add your HF Token as a secret")

In [ ]:
import requests
from transformers import AutoTokenizer
from huggingface_hub import HfApi

def diagnose_huggingface_error(hf_token):
    headers = {"Authorization": f"Bearer {hf_token}"}

    # Step 1: Check if the token is valid
    response = requests.get("https://huggingface.co/api/whoami-v2", headers=headers)
    if response.status_code == 401:
        return "Error: Invalid HuggingFace token. Please check if you are properly signed in and your token is correct."

    # Step 2: Check if you've been accepted by Meta - see instructions in last cell if not
    model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    response = requests.head(f"https://huggingface.co/{model_id}", headers=headers)
    if response.status_code == 403:
        return ("Error: Access denied to the model. "
                "Please ensure you have agreed to Meta's terms for the Llama model. "
                "Visit: https://huggingface.co/meta-llama")

    # Step 3: Check fine-grained permissions of the token
    api = HfApi()
    token_info = api.whoami(token=hf_token)
    if "scope" in token_info:
        scopes = token_info["scope"]
        if "read" not in scopes:
            return ("Error: Token does not have read access to external repos. "
                    "Ensure the token's fine-grained permissions include 'read'.")

    # Step 4: Test if model loading is allowed
    try:
        AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, token=hf_token)
        return "Success: All checks passed, and the model should load correctly."
    except Exception as e:
        return f"Unexpected error when loading the model: {e}"

result = diagnose_huggingface_error(hf_token)
print(result)

# If Meta has not approved your access to Llama 3.1

It's a mystery to me, but for some reason, occasionally Meta doesn't approve access. They don't give a reason.

If this happens to you, you can use Qwen 2.5 (or Qwen 3) as a swap-in replacement to Llama throughout this course!

But if you'd like to persist with Llama, then the following steps have worked with every student I've spoken with so far. Please let me know if this doesn't work for you!


1. Create a new gmail email (ugh) or use a spare if you have one
2. Create a new HF account via this new gmail and create a new WRITE token for your `.env` file and colab secrets
3. Apply for Llama 3.1 access (only 3.1, not 3.2)
4. For email, be sure to use this identical new gmail email address - be absolutely sure that the email matches precisely
5. For Organization Affiliation put `<Your Name> Personal Education` and for role put `AI Researcher`

Let me know! And remember, Qwen is just as good - if not better.


### A last resort! Submitting to Meta

A student (thank you Sergio!) has suggested this approach as a final last resort, which was successful for him:

___

If someone's access has been denied, here is a suggested alternative: go directly to the Meta webpage and request the model from there:

https://www.llama.com/llama-downloads/

1. Fill out the requested information and introduce an Organization/Affiliation. In my case, I used the name of my previous company. Students could perhaps use the name of their university or school.
2. Select the Llama model(s).
3. Once access granted, follow the instructions.

It is likely that users will need to replace this line:

`tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B', trust_remote_code=True)`

with the following:

```
!pip install llama-stack
!llama model list
!llama model download --source meta --model-id  Llama3.1-8B
tokenizer = AutoTokenizer.from_pretrained('/root/.llama/checkpoints/Llama3.1-8B', trust_remote_code=True)
```

Meta will likely provide a link that the user must paste during the download/authentication process.
